In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
text_file = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("C:\\Users\\sharolin\\Desktop\\entrada\\GuatemalaExportsTo.csv")
rddfiltro = text_file.rdd.map(tuple)
rddPAIS = rddfiltro.map(lambda word: (word[4],word[1]))
rddPAIS.take(5)

[('China', 18490000.0),
 ('China', 9830000.0),
 ('China', 8360000.0),
 ('China', 7110000.0),
 ('China', 4070000.0)]

In [3]:
rddCONTEO=rddPAIS.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCONTEO.collect())

Conteo total -> [('China', 64176766.0), ('Mexico', 511215129.0), ('Canada', 222575942.0), ('Japon', 164579976.0), ('United Kingdom', 101910872.0), ('Costa Rica', 428419775.0), ('Saudi Arabia', 111274535.0), ('Gabon', 6420.0), ('Italy', 166680831.0), ('France', 34023574.0), ('Brazil', 39146101.0), ('Germany', 144422819.0), ('Honduras', 967311616.0), ('Venezuela', 14998293.0), ('El Salvador', 1217138626.0), ('United States', 3803123168.0)]


In [4]:
rddORDEN = sc.parallelize(rddCONTEO.sortBy(lambda a: a[1],True).take(5))
print("Paises con menor valor de exportaciones -> %s" % rddORDEN.collect())

Paises con menor valor de exportaciones -> [('Gabon', 6420.0), ('Venezuela', 14998293.0), ('France', 34023574.0), ('Brazil', 39146101.0), ('China', 64176766.0)]


In [5]:
rddNombres = rddORDEN.map(lambda x: (x[0]))
rddTotales = rddORDEN.map(lambda x: (x[1]))
print(rddNombres.collect())
print(rddTotales.collect())

['Gabon', 'Venezuela', 'France', 'Brazil', 'China']
[6420.0, 14998293.0, 34023574.0, 39146101.0, 64176766.0]


In [6]:
fig = go.Figure(data=go.Pie(labels=rddNombres.collect(),values=rddTotales.collect()))
fig.update_layout(title_text='Paises con menor valor de exportaciones',title_font_size=30)
fig.update_traces(hoverinfo='label+percent', textinfo='value',textfont_size=28)
fig.write_html('R2_A1.html', auto_open=True)